In [1]:
import warnings

warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [7]:
titleX = []
spX = []
mrpX = []
discountX = []
linkX = []
quantityX = []
categoryX = []
ratingX = []
rating_countX = []
review_countX = []

for page in range(1, 10):
    
    # target URL to scrap
    url = f"https://www.jiomart.com/category/staples/dals-pulses/page/{page}"
    
    # headers
    headers = {
        'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
        }
    # send request to download the data
    response = requests.request("GET", url, headers=headers)

    # parse the downloaded data
    data = BeautifulSoup(response.text, 'html.parser')
    
    dal_data = data.find_all('div', attrs={'class', 'col-md-3 p-0'})
    
    print('Number of products : ', len(dal_data))

    # source code of hotel cards clsgetname
    for dal in dal_data:
        # Product name and Quantity
        dal_name_qty = dal.find('span', attrs={'class': 'clsgetname'})
        dal_name_qty_diff = re.findall('\d*\D+',dal_name_qty.text)
        titleX.append(dal_name_qty_diff[0])
        if len(dal_name_qty_diff) > 1:
            quantityX.append(dal_name_qty_diff[1])
        else:
            quantityX.append('N')
        
        # Fetch Price and regex to extract actual price and offered price
        ori_price = dal.find('span', attrs={'class', 'price-box'})
        num = re.findall("\d+\.\d+", ori_price.text)
        
        mrpX.append(num[0])
        spX.append(num[1])
        
        # Product Image Link
        image_link = dal.find('img', src=True)
#         image_link = image_link['src'])
        
        # Product Link
        product_link = dal.find('a', attrs={'class': 'category_name'})
        linkX.append(product_link['href'])

        discount_sec = dal.find('span', attrs={'class': 'dis_section'})
        discountX.append(discount_sec.text)
        
        # Handling unavailable data from the website
        categoryX.append('dals-pulses')
        ratingX.append(99)
        rating_countX.append(99)
        review_countX.append(99)

Number of products :  20
Number of products :  0
Number of products :  0
Number of products :  0
Number of products :  0
Number of products :  0
Number of products :  0
Number of products :  0
Number of products :  0


In [8]:
df = pd.DataFrame({'Title': titleX, 'Selling Price': spX, 'MRP': mrpX, 'Product Link': linkX, 'Quantity': quantityX,
                 'Discount': discountX, 'Rating': ratingX, 'Category': categoryX, 'Rating Count': rating_countX,
                 'Review Count': review_countX})

df.head()

,Title,Selling Price,MRP,Product Link,Quantity,Discount,Rating,Category,Rating Count,Review Count
0,Plain Toor Dal,190.00,280.00,https://www.jiomart.com/plain-toor-dal-2-kg-49...,2 kg,32% off,99,dals-pulses,99,99
1,Raw Peanuts,125.00,127.00,https://www.jiomart.com/raw-peanuts-1-kg-loose...,1 Kg (Loose),2% off,99,dals-pulses,99,99
2,Good Life Toor Dal,108.00,137.00,https://www.jiomart.com/good-life-toor-dal-1-k...,1 kg,21% off,99,dals-pulses,99,99
3,Good Life Toor Dal Oily,116.00,133.00,https://www.jiomart.com/good-life-toor-dal-oil...,1 kg,13% off,99,dals-pulses,99,99
4,Loose Toor Dal Value,95.00,101.00,https://www.jiomart.com/loose-toor-dal-value-1...,1 kg,6% off,99,dals-pulses,99,99


In [9]:
df.to_csv('reliance_dal.csv', index=False)